In [48]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" 
value="Click here to toggle on/off the code."></form>''')

# Inferential Analysis on Brand Impact Survey


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# this allows plots to appear directly in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14
# pip install patsy
# pip install statsmodels
import statsmodels.formula.api as smf
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

from IPython.core.display import display, HTML

In [8]:
# Transaction Data After ETL
url_transaction = '/Users/shaokuixing/Desktop/whatever/survey_delivery_event_inscreen20160522.csv'
trans = pd.read_csv(url_transaction, sep='\t', thousands = '.', decimal =',')
trans.sample(n=3,random_state=23)


,surveyid,answersetid,uid,deliveryid,adspaceid,apptype,apptypegroupid,iab_adspace_category,originaladid,iab_ad_category,adname,templatetype,size,video,streamview,storelocator,swipe,iosversion,androidversion,platform,modelid,screensize,screenwidth,browserid,browser,sdkclientid,sdkclientname,ipcity,contype,con,rss,click,firstinteraction,close,click_adjusted,close_adjusted,vtabove0pct,vtabove10pct,vtabove20pct,vtabove30pct,vtabove40pct,vtabove50pct,vtabove60pct,vtabove70pct,vtabove80pct,vtabove90pct,vtabove100pct
38126,447,347127,2372328936987073210,93907940568,3631,IPHONE_APP,MOBILE_APP,Arts & Entertainment,38658,Gambling/Casino,OSE002897 Casumo Januari Adsync Takeover - AdS...,swipe,takeover,0,0,0,1,19000,(null),ios,104772,4.7,750,7,Mobile Safari,1,iossdk,(null),lte,3,(null),0,0,1,0,1,1798,1798,1798,1798,1798,1798,1798,1798,1798,1798,1798
8060,498,394389,413866027122445885,97261102528,7526,MOBILE_WEB,MOBILE_WEB,Style & Fashion,39435,Food & Drink,"OFI001807 Bunge Finland Oy Bunge, Älva Ruotsi ...",takeover,takeover,0,0,0,0,19100,(null),ios,110834,4.7,750,7,Mobile Safari,5,dynamic,Eslöv,(null),3,(null),0,0,0,0,0,63,63,63,63,63,63,63,63,63,63,63
79930,316,235532,6834545996892502619,87254645084,1947,MOBILE_WEB,MOBILE_WEB,Sports,35772,U.S. Government Resources,OSE002723 Försvarsmakten Officer Takeover -Sta...,takeover,takeover,0,0,0,0,(null),25100,android,106563,5.0,1080,2,Chrome Mobile,5,dynamic,Stockholm,(null),0,(null),0,0,0,0,0,23571,23571,23571,23571,23571,23571,23571,23571,23571,23571,0


In [9]:
# row and column count
print ('no. rows and columns in total dataset :',trans.shape)

('no. rows and columns in total dataset :', (116718, 47))


In [14]:
# show all variables and its datatype
print 'Take a look at all variables and their data types:'
print trans.dtypes

Take a look at all variables and their data types:
surveyid                 int64
answersetid              int64
uid                      int64
deliveryid               int64
adspaceid                int64
apptype                 object
apptypegroupid          object
iab_adspace_category    object
originaladid             int64
iab_ad_category         object
adname                  object
templatetype            object
size                    object
video                    int64
streamview               int64
storelocator             int64
swipe                    int64
iosversion              object
androidversion          object
platform                object
modelid                 object
screensize              object
screenwidth             object
browserid                int64
browser                 object
sdkclientid              int64
sdkclientname           object
ipcity                  object
contype                 object
con                      int64
rss                

In [18]:
# Create Binary dummy variable on string objects
trans1 = trans.copy()
apptype_dum = pd.get_dummies(trans1['apptype'])
apptype_dum.head()

,ANDROID_APP,IPAD_WEB,IPHONE_APP,MOBILE_WEB
0,0,0,1,0
1,0,0,1,0
2,0,0,0,1
3,0,0,1,0
4,0,0,0,1


In [53]:
# Variables that are nominal data transformed into numeric categorical representation
# Instead of creating multiple dimensions
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(trans.apptypegroupid)
trans1['apptypegroup_dum']=le.transform(trans.apptypegroupid)
trans1['apptypegroup_dum']=trans1['apptypegroup_dum'].astype('category', ordered=False)
trans1[['apptypegroupid','apptypegroup_dum']].sample(5)
#astype('category', categories=['one','two','three'], ordered=True)

,apptypegroupid,apptypegroup_dum
111358,MOBILE_WEB,1
115663,MOBILE_WEB,1
33033,MOBILE_APP,0
94098,MOBILE_APP,0
585,MOBILE_APP,0


In [23]:
# find unique values in nominal variable
trans.apptypegroupid.unique()

array(['MOBILE_APP', 'MOBILE_WEB'], dtype=object)

In [52]:
# check if datatype of the new dummy variable
trans1.apptypegroup_dum.dtype.name

'category'

In [54]:
# Factorize apptype
trans1['apptype_dum']=pd.factorize(trans1.apptype)[0]
trans1['apptype_dum']=trans1['apptype_dum'].astype('category', ordered=False)
trans1[['apptype','apptype_dum']].sample(5)

,apptype,apptype_dum
68884,IPHONE_APP,0
69088,IPHONE_APP,0
70793,MOBILE_WEB,1
114808,IPHONE_APP,0
24237,MOBILE_WEB,1


In [56]:
# Factorize apptype
trans1['apptype_dum']=pd.factorize(trans1.apptype)[0]
trans1['apptype_dum']=trans1['apptype_dum'].astype('category', ordered=False)
trans1[['apptype','apptype_dum']].sample(5)

,apptype,apptype_dum
19446,IPHONE_APP,0
7033,MOBILE_WEB,1
6918,ANDROID_APP,2
15913,IPHONE_APP,0
57942,IPHONE_APP,0
